In [ ]:
#!pip install openpyxl # Uncomment to install openpyxl

# NOTE

### THIS IS A COPY OF FINAL OPTIMISATION ALGORITHM FROM MANDATORY TASK DIRECTORY

### Libraries

In [1]:
import pandas as pd
import numpy as np
import time

import PeriodMaximisation as pm
from Battery import Battery

%run ./MiniModels.ipynb

## Optimisation Algorithm

In [2]:
def ConvertFormat_BunToMeat(df):
    
    # Converting simple format to actual format
    df["Power"] = np.where(df["Status"] == "Charge", df["Actual"] * 2, df["Actual"] * 2 / 0.9)
    df = pm.renameColumns(df[["Time", "Price", "Power", "Actual", "Opening Capacity", "Closing Capacity"]])
    
    return df

def ConvertFormat_MeatToBun(act, df):
    
    # Converting actual format to simple format
    act = act[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)", 
               "Market Dispatch (MWh)", "Opening Capacity (MWh)", 
               "Closing Capacity (MWh)"]] # Select time and victoria prices
    act.columns = ["Time", "Price", "Actual", "Opening Capacity", "Closing Capacity"] # Rename columns
    act["Time"] = pd.to_datetime(df["Time"]) # Convert data type
    act = act.sort_values("Time").reset_index(drop = True) # Finalise
    act["Restrict"] = - np.where(act["Actual"] <= 0, act["Actual"] * 0.9, act["Actual"] / 0.9)
    act["Status"] = np.where(act["Actual"] < 0, "Charge", np.where(act["Actual"] > 0, "Discharge", "Do Nothing"))

    sim = pd.merge(df["Time"], act.drop("Time", axis = 1), left_index = True, right_index = True)

    return sim

In [6]:
def TopBun(df, n = 17):
    Model = MovingAverage(n, df) # Moving Average
    Model = Maximisation(Model, df) # Region Maximisation
    Model = LossRemoval(Model) # Loss Removal
    Model = Stationary(Model, df) # Stationary Maximisation
    Model = ShiftAction(Model, df) # Action Shift
    
    return Model

def Meat(df):
    tmp = df.copy()
    # Converting format
    tmp = ConvertFormat_BunToMeat(tmp)
    
    # Get time period
    time = pm.getTimePeriod(tmp)
    
    # Run Dependendy Optimisation Algorithm
    tmp = pm.DependencyOptimisation(tmp, time) # PeriodOptimisation
    
    # Convert back format
    tmp = ConvertFormat_MeatToBun(tmp, df)
    
    return tmp

def LowerBun(Model, df):
    
    Model = Maximisation(Model, df = df, Chronos = False) # Region Maximisation
    Model = Stationary(Model, df = df) # Stationary Maximisation
    Model = ShiftAction(Model, df = df) # Action Shift

    return Model

def SandwichModel(original_data):
    tmp = original_data.copy()
    
    start = time.time()
    print("Calculating First Step")
    tmp = TopBun(tmp, n = 17)
    first = time.time()
    print(f"It took {first - start} seconds for First Step")
    
    print("#######################\n")
    
    print("Calculating Second Step")
    tmp = Meat(tmp)
    second = time.time()
    print(f"It took {second - first} seconds for Second Step")
    
    print("#######################\n")
    
    print("Calculating Final Step")
    tmp = LowerBun(tmp, original_data)
    third = time.time()
    print(f"It took {third - second} seconds for Third Step")
    total = time.time()
    
    print("#######################\n")
    
    print(f"In total cooking time took {total - start} seconds")
    tmp = ConvertFormat_BunToMeat(tmp)
    return tmp